## This short introduction uses Keras to:
1. Build a neural network that classifies images
2. Train this neural network
3. And, finally, evaluate the accuracy of the model

In [1]:
import tensorflow as tf

2021-07-12 13:56:05.261078: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-12 13:56:05.261104: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Load and prepare the MMNIST dataset. Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


Build the ```tf.keras.Sequential``` model by stacking layers. Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

2021-07-12 14:00:21.663202: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-12 14:00:21.663236: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-12 14:00:21.663261: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (leno-lab): /proc/driver/nvidia/version does not exist
2021-07-12 14:00:21.663490: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()

In [5]:
predictions

array([[  48.49642 ,   23.90725 ,   61.895542,  -34.274017, -191.26398 ,
          30.574589,  -71.373825, -157.33699 ,  166.43184 ,  -75.399345]],
      dtype=float32)

The ```tf.nn.softmax``` function converts these logits to "probabilities" for each class:

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

#### Note: It is possible to bake this ```tf.nn.softmax``` in as the activation function for the last layer of the network. While this can me the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

The ```losses.SparseCategoricalCrossentropy``` loss takes a vector of logits and a ```True``` index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class, so the initial loss should be close to ```-tf.math.log(1/10) ~= 2.3```

In [8]:
loss_fn(y_train[:1], predictions).numpy()

135.85725

In [9]:
model.compile(optimizer='adam',
             loss=loss_fn,
             metrics=['accuracy'])

The ```model.fit``` method adjusts the model parameters to minimize the loss:

In [10]:
model.fit(x_train, y_train, epochs=5)

2021-07-12 14:14:09.489983: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-12 14:14:09.490380: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3193550000 Hz


Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 2.6652 - accuracy: 0.7614
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.6137 - accuracy: 0.8402
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4961 - accuracy: 0.8707
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4225 - accuracy: 0.8898
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3838 - accuracy: 0.9009


The ```model.evaluate``` method checks the models performance, usually on a "Validation set" or "Test set"

In [11]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.2959 - accuracy: 0.9374


[0.2958924472332001, 0.9373999834060669]